In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopy.distance
import geopy.point

In [5]:
df_skiers = pd.read_csv("../data/filtered/Dati_filtrati_all.csv")
df_skiers = df_skiers[df_skiers["DNF"] == False]

In [6]:
ranges = [50, 55, 60, 65, 70, 75]
columns = ["Athlete", "Day"]
for i_range, range_ in enumerate(ranges):
    if i_range == 0:
        columns.append(f"0 - {range_}")
    elif i_range == len(ranges) - 1:
        columns.append(f"{ranges[i_range - 1]} - {range_}")
        columns.append(f">{range_}")
        columns.append("File_gps")
    else:
        columns.append(f"{ranges[i_range - 1]} - {range_}")

vel_df = pd.DataFrame(columns=columns)

for day in ["G1", "P1", "P2"]:
    df_skiers_day = df_skiers[df_skiers["Day"] == day]
    n_intervals = len(ranges) + 1
    global_intervals = [0] * n_intervals

    for athlete, skier_file, startTime, duration in zip(df_skiers_day["Atleta"], df_skiers_day["File_gps"], df_skiers_day["StartTime"], df_skiers_day["Time"]):
        df_skier = pd.read_csv(skier_file)
        df_skier_started = df_skier[df_skier["Timestamp"] >= startTime]
        df_skier_run = df_skier_started[df_skier_started["Timestamp"] <= startTime + duration]
        df_skier_run = df_skier_run.sort_values('Timestamp', ascending=False)
        intervals = [0] * n_intervals

        for velocity, long1, lat1, alt1, long2, lat2, alt2 in zip(df_skier_run["GNSS_SpeedOverGround"][:-1], df_skier_run["GNSS_Longitude"][:-1], df_skier_run["GNSS_Latitude"][:-1], df_skier_run["GNSS_Altitude"][:-1],df_skier_run["GNSS_Longitude"][1:], df_skier_run["GNSS_Latitude"][1:],df_skier_run["GNSS_Altitude"][1:]):

            distance2d = geopy.distance.geodesic((long1, lat1), (long2, lat2)).m
            distance = np.sqrt(distance2d**2 + (alt1 - alt2)**2)

            for i_range in range(n_intervals):
                if i_range == n_intervals - 1 or velocity <= ranges[i_range]:
                    intervals[i_range] += distance
                    global_intervals[i_range] += distance
                    break
        plt.xlabel("Velocity (km/h)")
        plt.ylabel("Distance travelled (m)")
        plt.title(f"Travelled distance in different velocity ranges of skier {athlete} on day {day}")
        plt.savefig(f"../figures/Travelled_distance_in_different_velocity_ranges_of_skier_{athlete}_on_day_{day}.png")
        plt.close()

        intervals.insert(0, day)
        intervals.insert(0, athlete)
        intervals.append(skier_file)
        vel_df.loc[len(vel_df)] = intervals


    plt.bar(range(1, n_intervals + 1), global_intervals)
    plt.xlabel("Velocity (km/h)")
    plt.ylabel("Distance travelled (m)")
    plt.title(f"Travelled distance in different velocity ranges of skiers on day {day}")
    plt.savefig(f"../figures/Travelled_distance_in_different_velocity_ranges_of_skiers_on_day_{day}.png")
    plt.close()
vel_df.to_csv("../data/filtered/velocity_ranges.csv", index=False)